<a href="https://colab.research.google.com/github/bialyy10/Uczenie_maszynowe_w_finansach_1/blob/main/BD_spark_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Big Data

### Prowadzący, modyfikacje i aktualizacja: dr Marcin Szmydt

Opracowanie pierwotne: mgr inż. Dominik Filipiak
Notebook do zdalnej pracy ze Sparkiem.

### Przygotowanie sparka do pracy z colabem

Przed przystąpieniem do pracy musimy uruchomić poniższe komórki.
W razie kłopotów sprawdźcie czy nie zmieniła się wersja tutaj: http://www.apache.org/dist/spark/. W razie czego trzeba pozmieniać numerki wszędzie (np. 3.4.1 na 3.5.1).

Tu ściągamy Javę oraz Sparka razem z Hadoopem, a także instalujemy pakiet findspark w pythonie. Pewnie już się domyślacie, że przez wykrzyknik można wywołać linuksowe polecenia w Colabie.



In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/spark-3.5.1/spark-3.5.1-bin-hadoop3.tgz
!tar xf spark-3.5.1-bin-hadoop3.tgz
!pip install -q findspark==1.3.0

Instalujemy pyspark w Pythonie

In [2]:
!pip install -q pyspark==3.5.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


Tu ustalamy zmienne środowiskowe.

In [3]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.1-bin-hadoop3"

Jesteśmy gotowi do odpalenia Sparka - tworzymy sesję i kontekst.

In [4]:
import findspark
import pyspark
from pyspark.sql import SparkSession
from pyspark import SparkContext

findspark.init("spark-3.5.1-bin-hadoop3")
sc = pyspark.SparkContext('local[*]') #gwiazka oznacza że wykorzystujemy wszystkie rdzenie na wszystkich maszynach do obliczeń
spark = SparkSession.builder.appName('abc').getOrCreate()

In [ ]:
!free -mh # Komenda na sprawdzenie dostępnej pamięci RAM

               total        used        free      shared  buff/cache   available
Mem:            12Gi       981Mi       2.0Gi       1.0Mi       9.7Gi        11Gi
Swap:             0B          0B          0B


# Możemy przystąpić do pracy!

In [ ]:
sc.parallelize([1,2,3]).map(lambda x: x + 1).collect()

[2, 3, 4]

# Zajęcia 25.03.2024

In [ ]:
sc

<SparkContext master=local[*] appName=pyspark-shell>

###Przechodzenie między środowiskiem lokalnym (Python) a środowiskiem rozproszonym (Spark)

In [ ]:
lista = [1, 2, 3]

In [ ]:
type(lista)

list

In [ ]:
distributed_list = sc.parallelize(lista) #przenoszenie do świata rozproszonego (przechodzi z lokalnego phtyona do rdd z klastra)

In [ ]:
type(distributed_list)

pyspark.rdd.RDD

In [ ]:
distributed_list.take(10) # wywołanie x elementów

[1, 2, 3]

In [ ]:
listapython = distributed_list.map(lambda x: x + 1).collect() #przenoszenie z świata(maszyny) rozproszonego do lokalnego phytona

In [ ]:
#lampda x: x+1 inny sposób definiowania funkcji zamiast def funk.. używamy wtedy kiedy dana funkcja jest wykorzystywana tylko raz
#np lampda x: x*10 mnożenie każdego elementu przez 10

In [ ]:
listapython

[2, 3, 4]

In [ ]:
type(listapython)

list

###RDD -Wczytywanie plików, podstawowe operacje (take, map, filter)

In [ ]:
!head -5 baby.csv

Year,First Name,County,Sex,Count
2013,DAVID,KINGS,M,272
2013,JAYDEN,KINGS,M,268
2013,JAYDEN,QUEENS,M,219
2013,MOSHE,KINGS,M,219


In [ ]:
baby_names = sc.textFile('baby.csv') #ładowanie rdd

In [ ]:
baby_names.take(10)

['Year,First Name,County,Sex,Count',
 '2013,DAVID,KINGS,M,272',
 '2013,JAYDEN,KINGS,M,268',
 '2013,JAYDEN,QUEENS,M,219',
 '2013,MOSHE,KINGS,M,219',
 '2013,ETHAN,QUEENS,M,216',
 '2013,SOPHIA,QUEENS,F,204',
 '2013,DANIEL,QUEENS,M,203',
 '2013,JACOB,KINGS,M,202',
 '2013,ESTHER,KINGS,F,196']

In [ ]:
rows = baby_names.map(lambda x: x.split(','))

In [ ]:
rows.take(3)

[['Year', 'First Name', 'County', 'Sex', 'Count'],
 ['2013', 'DAVID', 'KINGS', 'M', '272'],
 ['2013', 'JAYDEN', 'KINGS', 'M', '268']]

In [ ]:
rows.filter(lambda x: 'Year' not in x).take(5)

[['2013', 'DAVID', 'KINGS', 'M', '272'],
 ['2013', 'JAYDEN', 'KINGS', 'M', '268'],
 ['2013', 'JAYDEN', 'QUEENS', 'M', '219'],
 ['2013', 'MOSHE', 'KINGS', 'M', '219'],
 ['2013', 'ETHAN', 'QUEENS', 'M', '216']]

In [ ]:
rows_filter = rows.filter(lambda x: 'Year' not in x)

In [ ]:
rows_filter.take(5)

[['2013', 'DAVID', 'KINGS', 'M', '272'],
 ['2013', 'JAYDEN', 'KINGS', 'M', '268'],
 ['2013', 'JAYDEN', 'QUEENS', 'M', '219'],
 ['2013', 'MOSHE', 'KINGS', 'M', '219'],
 ['2013', 'ETHAN', 'QUEENS', 'M', '216']]

###Zadanie 1 - Jak odfiltrować z RDD imiona które które mają mniej niż 250 wystąpień (count)

In [ ]:
#kod

In [ ]:
rows.take(5)

[['Year', 'First Name', 'County', 'Sex', 'Count'],
 ['2013', 'DAVID', 'KINGS', 'M', '272'],
 ['2013', 'JAYDEN', 'KINGS', 'M', '268'],
 ['2013', 'JAYDEN', 'QUEENS', 'M', '219'],
 ['2013', 'MOSHE', 'KINGS', 'M', '219']]

In [ ]:
(rows.filter(lambda x: 'Year' not in x)
     .filter(lambda x: int(x[4])<250).take(10))


[['2013', 'JAYDEN', 'QUEENS', 'M', '219'],
 ['2013', 'MOSHE', 'KINGS', 'M', '219'],
 ['2013', 'ETHAN', 'QUEENS', 'M', '216'],
 ['2013', 'SOPHIA', 'QUEENS', 'F', '204'],
 ['2013', 'DANIEL', 'QUEENS', 'M', '203'],
 ['2013', 'JACOB', 'KINGS', 'M', '202'],
 ['2013', 'ESTHER', 'KINGS', 'F', '196'],
 ['2013', 'ETHAN', 'KINGS', 'M', '196'],
 ['2013', 'ISABELLA', 'QUEENS', 'F', '192'],
 ['2013', 'DANIEL', 'KINGS', 'M', '188']]

###Cache (i wpływ na czas przetarzania)

In [ ]:
nums = sc.parallelize(range(1, 1000001))

In [ ]:
nums.count()

1000000

In [ ]:
def isPrime(n) :

    # Corner cases
    if (n <= 1) :
        return False
    if (n <= 3) :
        return True

    # This is checked so that we can skip
    # middle five numbers in below loop
    if (n % 2 == 0 or n % 3 == 0) :
        return False

    i = 5
    while(i * i <= n) :
        if (n % i == 0 or n % (i + 2) == 0) :
            return False
        i = i + 6

    return True

In [ ]:
isPrime(13)

True

In [ ]:
primes = nums.filter(isPrime)

In [ ]:
primes.count()

78498

In [ ]:
import time

In [ ]:
start = time.time()
print(f'Jest {primes.count()} liczb pierwszych w milionie')
end = time.time()
print(f'Upłynęło {end-start} sekund')

Jest 78498 liczb pierwszych w milionie
Upłynęło 3.2738828659057617 sekund


In [ ]:
primes.cache() #to tez jest transformacja

PythonRDD[19] at RDD at PythonRDD.scala:53

In [ ]:
start = time.time()
print(f'Jest {primes.count()} liczb pierwszych w milionie') #wykonanie akcji, dlatego trwa dłużej
end = time.time()
print(f'Upłynęło {end-start} sekund')

Jest 78498 liczb pierwszych w milionie
Upłynęło 5.451779365539551 sekund


In [ ]:
start = time.time()
print(f'Jest {primes.count()} liczb pierwszych w milionie') #uzyte 2 raz, wynik szybszy bo zaoamiętał w cashe operacje
end = time.time()
print(f'Upłynęło {end-start} sekund')

Jest 78498 liczb pierwszych w milionie
Upłynęło 0.6099245548248291 sekund


In [ ]:
start = time.time()
print(f'Jest {primes.count()} liczb pierwszych w milionie')
end = time.time()
print(f'Upłynęło {end-start} sekund')

Jest 78498 liczb pierwszych w milionie
Upłynęło 0.3792726993560791 sekund


In [ ]:
primes.unpersist()

PythonRDD[19] at RDD at PythonRDD.scala:53

In [ ]:
start = time.time()
print(f'Jest {primes.count()} liczb pierwszych w milionie')
end = time.time()
print(f'Upłynęło {end-start} sekund')

Jest 78498 liczb pierwszych w milionie
Upłynęło 4.069396257400513 sekund


###Sampling

In [ ]:
one_to_ten = range(1,11)
parallel = sc.parallelize(one_to_ten)

In [ ]:
parallel.take(100)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [ ]:
?parallel.sample

In [ ]:
parallel.sample(True, .2).take(5)

[1, 3, 8, 9]

In [ ]:
parallel.takeSample(True, 2)

[9, 9]

In [ ]:
parallel_big = sc.parallelize(range(1,100001))

In [ ]:
parallel_big.sample(True, .2).count()

20092

In [ ]:
parallel_big.sample(True, .2, 2137).take(5)

[7, 8, 24, 24, 28]

###Intersection, Union, Joins

In [ ]:
one = sc.parallelize(range(1,10))
two = sc.parallelize(range(5,15))

In [ ]:
one.collect()

[1, 2, 3, 4, 5, 6, 7, 8, 9]

In [ ]:
two.collect()

[5, 6, 7, 8, 9, 10, 11, 12, 13, 14]

In [ ]:
one.intersection(two).collect()

[8, 5, 9, 6, 7]

In [ ]:
one.union(two).collect()

[1, 2, 3, 4, 5, 6, 7, 8, 9, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]

In [ ]:
one.union(two).distinct().collect()

[4, 8, 12, 1, 5, 9, 13, 2, 6, 10, 14, 3, 7, 11]

In [ ]:
names1 = sc.parallelize(('abe', 'abby', 'apple')).map(lambda x: (x, 1))
names2 = sc.parallelize(('apple', 'beatty', 'beatrice')).map(lambda x: (x, 5))
print(f'names1: {names1.collect()}')
print(f'names2: {names2.collect()}')

names1: [('abe', 1), ('abby', 1), ('apple', 1)]
names2: [('apple', 5), ('beatty', 5), ('beatrice', 5)]


In [ ]:
names1.join(names2).collect()

[('apple', (1, 5))]

In [ ]:
names1.leftOuterJoin(names2).collect()

[('abby', (1, None)), ('abe', (1, None)), ('apple', (1, 5))]

In [ ]:
names1.rightOuterJoin(names2).collect()

[('beatty', (None, 5)), ('beatrice', (None, 5)), ('apple', (1, 5))]

In [ ]:
names1.cartesian(names2).collect()

[(('abe', 1), ('apple', 5)),
 (('abe', 1), ('beatty', 5)),
 (('abe', 1), ('beatrice', 5)),
 (('abby', 1), ('apple', 5)),
 (('apple', 1), ('apple', 5)),
 (('abby', 1), ('beatty', 5)),
 (('abby', 1), ('beatrice', 5)),
 (('apple', 1), ('beatty', 5)),
 (('apple', 1), ('beatrice', 5))]

###Zapis do jednego pliku (użycie coalesce)

In [ ]:
wynik = names1.cartesian(names2)

In [ ]:
type(wynik)

pyspark.rdd.RDD

In [ ]:
wynik.saveAsTextFile('wynik.txt') #wynik rozproszony na 4 pliki, ponieważ maszyna posiada 4 rdzenie

In [ ]:
!cat wynik.txt/part-00000

(('abe', 1), ('apple', 5))


In [ ]:
wynik.coalesce(1).saveAsTextFile('wynik_one_partition.txt') #wywołanie do określonej liczby plików wyniku

In [ ]:
!cat wynik_one_partition.txt/part-00000

(('abe', 1), ('apple', 5))
(('abe', 1), ('beatty', 5))
(('abe', 1), ('beatrice', 5))
(('abby', 1), ('apple', 5))
(('apple', 1), ('apple', 5))
(('abby', 1), ('beatty', 5))
(('abby', 1), ('beatrice', 5))
(('apple', 1), ('beatty', 5))
(('apple', 1), ('beatrice', 5))


###Zadanie 2
1. Stwórz zmienną w rozproszonym środowisku Sparka (RDD), która zawiera liczby od jeden do miliona
2. Usuń z tej zmiennej liczby, które mają resztę z dzielenia przez 5 mniejszą niż 2
3. Wylosuj 1% rekordów z uzyskanego RDD bez zwracania z ziarnem losowości 12345
4. Zapisz na lokalnym systemie plików do jednego pliku tekstowego
5. Odczytaj pierwsze 3 liczby z zapisanego pliku

In [ ]:
#kod
milion = sc.parallelize(range(1,1000001))


In [ ]:
def dzielenie(n):
  return n % 5 >= 2


In [ ]:
zad = milion.filter(dzielenie)


In [ ]:
losowanie = zad.sample(False, .01, 12345)


In [ ]:
losowanie.coalesce(1).saveAsTextFile('losowanie6.txt')

In [ ]:
!head -3 /content/losowanie6.txt/part-00000

34
79
233


###groupByKey

In [ ]:
!head -5 baby.csv

Year,First Name,County,Sex,Count
2013,DAVID,KINGS,M,272
2013,JAYDEN,KINGS,M,268
2013,JAYDEN,QUEENS,M,219
2013,MOSHE,KINGS,M,219


In [ ]:
baby_names = sc.textFile('baby.csv')

In [ ]:
baby_names_no_header = baby_names.filter(lambda x: x!='Year,First Name,County,Sex,Count')

In [ ]:
baby_names_no_header.take(10)

['2013,DAVID,KINGS,M,272',
 '2013,JAYDEN,KINGS,M,268',
 '2013,JAYDEN,QUEENS,M,219',
 '2013,MOSHE,KINGS,M,219',
 '2013,ETHAN,QUEENS,M,216',
 '2013,SOPHIA,QUEENS,F,204',
 '2013,DANIEL,QUEENS,M,203',
 '2013,JACOB,KINGS,M,202',
 '2013,ESTHER,KINGS,F,196',
 '2013,ETHAN,KINGS,M,196']

In [ ]:
rows = baby_names_no_header.map(lambda x: x.split(','))

In [ ]:
rows.take(10)

[['2013', 'DAVID', 'KINGS', 'M', '272'],
 ['2013', 'JAYDEN', 'KINGS', 'M', '268'],
 ['2013', 'JAYDEN', 'QUEENS', 'M', '219'],
 ['2013', 'MOSHE', 'KINGS', 'M', '219'],
 ['2013', 'ETHAN', 'QUEENS', 'M', '216'],
 ['2013', 'SOPHIA', 'QUEENS', 'F', '204'],
 ['2013', 'DANIEL', 'QUEENS', 'M', '203'],
 ['2013', 'JACOB', 'KINGS', 'M', '202'],
 ['2013', 'ESTHER', 'KINGS', 'F', '196'],
 ['2013', 'ETHAN', 'KINGS', 'M', '196']]

In [ ]:
#W jakich hrabstwach występuje dane imię

In [ ]:
(rows
 .map(lambda x: (x[1], x[2]))
 .groupByKey()
 .map(lambda x: {x[0]: list(x[1])})
 ).collect()

[{'JAYDEN': ['KINGS', 'QUEENS']},
 {'ETHAN': ['QUEENS', 'KINGS']},
 {'SOPHIA': ['QUEENS']},
 {'DANIEL': ['QUEENS', 'KINGS']},
 {'ESTHER': ['KINGS']},
 {'ISABELLA': ['QUEENS']},
 {'AIDEN': ['KINGS']},
 {'NOAH': ['KINGS']},
 {'JOSEPH': ['KINGS']},
 {'DAVID': ['KINGS']},
 {'MOSHE': ['KINGS']},
 {'JACOB': ['KINGS', 'QUEENS']},
 {'LEAH': ['KINGS']},
 {'MATTHEW': ['QUEENS']}]

###reduceByKey

In [ ]:
rows.take(20)

[['2013', 'DAVID', 'KINGS', 'M', '272'],
 ['2013', 'JAYDEN', 'KINGS', 'M', '268'],
 ['2013', 'JAYDEN', 'QUEENS', 'M', '219'],
 ['2013', 'MOSHE', 'KINGS', 'M', '219'],
 ['2013', 'ETHAN', 'QUEENS', 'M', '216'],
 ['2013', 'SOPHIA', 'QUEENS', 'F', '204'],
 ['2013', 'DANIEL', 'QUEENS', 'M', '203'],
 ['2013', 'JACOB', 'KINGS', 'M', '202'],
 ['2013', 'ESTHER', 'KINGS', 'F', '196'],
 ['2013', 'ETHAN', 'KINGS', 'M', '196'],
 ['2013', 'ISABELLA', 'QUEENS', 'F', '192'],
 ['2013', 'DANIEL', 'KINGS', 'M', '188'],
 ['2013', 'JACOB', 'QUEENS', 'M', '186'],
 ['2013', 'AIDEN', 'KINGS', 'M', '182'],
 ['2013', 'LEAH', 'KINGS', 'F', '179'],
 ['2013', 'NOAH', 'KINGS', 'M', '168'],
 ['2013', 'JOSEPH', 'KINGS', 'M', '166'],
 ['2013', 'MATTHEW', 'QUEENS', 'M', '161']]

In [ ]:
#Ile razy dane imię wystąpiło we wszystkich hrabstwach

In [ ]:
?rows.reduceByKey()

Object `rows.reduceByKey()` not found.


In [ ]:
(rows
 .map(lambda x: (x[1],int(x[4])))
 .reduceByKey(lambda x,y: x+y)
).collect()

[('JAYDEN', 487),
 ('ETHAN', 412),
 ('SOPHIA', 204),
 ('DANIEL', 391),
 ('ESTHER', 196),
 ('ISABELLA', 192),
 ('AIDEN', 182),
 ('NOAH', 168),
 ('JOSEPH', 166),
 ('DAVID', 272),
 ('MOSHE', 219),
 ('JACOB', 388),
 ('LEAH', 179),
 ('MATTHEW', 161)]

###Reduce + flatMap

In [ ]:
x1 = sc.parallelize(['kot','pies','zajac', 'niedzwiedz'])
x2 = sc.parallelize(['wilk','pies','dzik', 'bocian'])

In [ ]:
x1.reduce(lambda x,y: x + '_' + y)

'kot_pies_zajac_niedzwiedz'

In [ ]:
#łączna liczba znaków we wszystkich wyrazach w x1

In [ ]:
(x1
 .map(lambda x: [x, len(x)])
 .flatMap(lambda x: [x[1]])
).reduce(lambda x, y: x + y)

22

# Zajęcia 12.04.2023

###CountByKey

In [ ]:
x12 = x1.union(x2)
x12.take(10)

['kot', 'pies', 'zajac', 'niedzwiedz', 'wilk', 'pies', 'dzik', 'bocian']

In [ ]:
(x12
 .map(lambda x: (x, 'wartosc-wymagane-pole'))
 ).collect()

[('kot', 'wartosc-wymagane-pole'),
 ('pies', 'wartosc-wymagane-pole'),
 ('zajac', 'wartosc-wymagane-pole'),
 ('niedzwiedz', 'wartosc-wymagane-pole'),
 ('wilk', 'wartosc-wymagane-pole'),
 ('pies', 'wartosc-wymagane-pole'),
 ('dzik', 'wartosc-wymagane-pole'),
 ('bocian', 'wartosc-wymagane-pole')]

In [ ]:
(x12
 .map(lambda x: (x, 'wartosc-wymagane-pole'))
 ).countByKey()

defaultdict(int,
            {'kot': 1,
             'pies': 2,
             'zajac': 1,
             'niedzwiedz': 1,
             'wilk': 1,
             'dzik': 1,
             'bocian': 1})

In [ ]:
slownik = (x12
 .map(lambda x: (x, 'wartosc-wymagane-pole'))
 ).countByKey()

In [ ]:
x12_dict_python = dict(slownik)
x12_dict_python

{'kot': 1,
 'pies': 2,
 'zajac': 1,
 'niedzwiedz': 1,
 'wilk': 1,
 'dzik': 1,
 'bocian': 1}

In [ ]:
x12_dict_python['pies']

2

###Broadcast

In [ ]:
my_local_dict = {'cityX': [1,2,3],
                 'cityY': [4,5,6]}

In [ ]:
my_rdd = sc.parallelize(['cityX','cityY'])
my_rdd.collect()

['cityX', 'cityY']

In [ ]:
my_rdd.map(lambda x: (x, my_local_dict[x][2])).collect() #Złe podejście

[('cityX', 3), ('cityY', 6)]

In [ ]:
my_broadcasted_dict = sc.broadcast(my_local_dict)

In [ ]:
my_broadcasted_dict.value

{'cityX': [1, 2, 3], 'cityY': [4, 5, 6]}

In [ ]:
my_rdd.map(lambda x: (x, my_broadcasted_dict.value[x][2])).collect() #Dobre podejście

[('cityX', 3), ('cityY', 6)]

###Accumulators

In [ ]:
accumulator = sc.accumulator(0)

In [ ]:
accumulator.value

In [ ]:
sc.parallelize([1,2,3,4]).foreach(lambda x: accumulator.add(x))

In [ ]:
accumulator.value

10

###Zadanie 3 - wordcount
1. Wczytaj plik TREASURE_ISLAND.txt z moodle do RDD
2. wykonaj wordcount (jakie słowo ile razy wystąpiło)

In [ ]:
#kod
!head -5 TREASURE_ISLAND.txt

TREASURE ISLAND

 To
S.L.O.,
an American gentleman


In [ ]:
count = (sc.textFile('TREASURE_ISLAND.txt', 1)
          . flatMap (lambda x : x.split (' '))
          . map( lambda x : ( x , 1 ) )
          . reduce By Key ( lambda x , y : x + y)
          .saveAsTextFile('output'))

SyntaxError: invalid syntax. Perhaps you forgot a comma? (<ipython-input-21-8b13947ec499>, line 1)

In [ ]:
plik = sc.textFile('TREASURE_ISLAND.txt')

In [ ]:
(plik
  .flatMap(lambda x : x.split(' '))
  .filter(lambda x : x != '')
  .map(lambda x : x.lower())
  .map(lambda x : (x, 1))
  .reduceByKey(lambda x,y: x+y)
 ).take(10)

[('treasure', 7),
 ('s.l.o.,', 1),
 ('an', 30),
 ('gentleman', 2),
 ('in', 222),
 ('whose', 1),
 ('following', 5),
 ('narrative', 5),
 ('designed,', 1),
 ('is', 44)]

##Spark SQL (DataFrames)

In [ ]:
!head -5 people.txt

Name,Age
Michael, 29
Andy, 30
Justin, 19
Filip, 99


In [ ]:
spark

In [ ]:
df = spark.read.csv('people.txt', header=True)

In [ ]:
type(df)

pyspark.sql.dataframe.DataFrame

In [ ]:
df.show(100)

+-------+---+
|   Name|Age|
+-------+---+
|Michael| 29|
|   Andy| 30|
| Justin| 19|
|  Filip| 99|
+-------+---+



In [ ]:
df.storageLevel.useMemory

False

In [ ]:
df.cache()

DataFrame[Name: string, Age: string]

In [ ]:
df.storageLevel.useMemory

True

In [ ]:
import pyspark.sql.functions as f

In [ ]:
x = df.select('Age', df['Age']+2, f.col('Age')+2).toPandas() #3 sposoby na modyfikację kolumny

In [ ]:
x

,Age,(Age + 2),(Age + 2)
0,29,31.0,31.0
1,30,32.0,32.0
2,19,21.0,21.0
3,99,101.0,101.0


In [ ]:
df.show(100)

+-------+---+
|   Name|Age|
+-------+---+
|Michael| 29|
|   Andy| 30|
| Justin| 19|
|  Filip| 99|
+-------+---+



In [ ]:
(df
 .filter(f.col('Age') > 20)
 .filter(f.col('Name').startswith('F'))
 ).show(100)

+-----+---+
| Name|Age|
+-----+---+
|Filip| 99|
+-----+---+



In [ ]:
df.groupBy('Name').count().show()

+-------+-----+
|   Name|count|
+-------+-----+
|Michael|    1|
|  Filip|    1|
|   Andy|    1|
| Justin|    1|
+-------+-----+



###TempView / GlobalTempView (ala SQL)

In [ ]:
df.createOrReplaceTempView('people')
sqlDF = spark.sql("SELECT * FROM people WHERE Age>20 AND SUBSTR(Name,1,1)='F'")
sqlDF.show()

+-----+---+
| Name|Age|
+-----+---+
|Filip| 99|
+-----+---+



In [ ]:
df.createOrReplaceGlobalTempView('people_g')
sqlDF = spark.sql("SELECT * FROM global_temp.people_g WHERE Age>20 AND SUBSTR(Name,1,1)='F'")
sqlDF.show()

+-----+---+
| Name|Age|
+-----+---+
|Filip| 99|
+-----+---+



### rdd $\mapsto$ DF

In [ ]:
lines = sc.textFile('people_noheader.txt')

In [ ]:
lines.take(10)

['Michael, 29', 'Andy, 30', 'Justin, 19', 'Filip, 99']

In [ ]:
parts = lines.map(lambda x: x.split(','))
parts.take(10)

[['Michael', ' 29'], ['Andy', ' 30'], ['Justin', ' 19'], ['Filip', ' 99']]

In [ ]:
parts = parts.map(lambda x: [x[0], int(x[1].strip())])
parts.take(10)

[['Michael', 29], ['Andy', 30], ['Justin', 19], ['Filip', 99]]

In [ ]:
from pyspark.sql import Row

In [ ]:
people = parts.map(lambda x: Row(Name=x[0], Age=x[1]))
people.take(10)

[Row(Name='Michael', Age=29),
 Row(Name='Andy', Age=30),
 Row(Name='Justin', Age=19),
 Row(Name='Filip', Age=99)]

In [ ]:
type(people)

pyspark.rdd.PipelinedRDD

In [ ]:
DFpeople = spark.createDataFrame(people)

In [ ]:
type(DFpeople)

pyspark.sql.dataframe.DataFrame

###modyfikacja schema - sposob 1

In [ ]:
DFpeople.show()

+-------+---+
|   Name|Age|
+-------+---+
|Michael| 29|
|   Andy| 30|
| Justin| 19|
|  Filip| 99|
+-------+---+



In [ ]:
DFpeople.printSchema()

root
 |-- Name: string (nullable = true)
 |-- Age: long (nullable = true)



In [ ]:
from pyspark.sql.types import *

In [ ]:
fields = [StructField('Names', StringType()),
          StructField('Age', IntegerType())]

In [ ]:
schema = StructType(fields)

In [ ]:
schema

StructType([StructField('Names', StringType(), True), StructField('Age', IntegerType(), True)])

In [ ]:
DFpeople = spark.createDataFrame(people, schema)

In [ ]:
DFpeople.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: integer (nullable = true)



In [ ]:
DFpeople.show()

+-------+---+
|  Names|Age|
+-------+---+
|Michael| 29|
|   Andy| 30|
| Justin| 19|
|  Filip| 99|
+-------+---+



In [ ]:
DFpeople.createOrReplaceTempView('people')

In [ ]:
x = spark.sql("SELECT * FROM people WHERE AGE >=20 AND AGE <=40")

In [ ]:
x.show()

+-------+---+
|  Names|Age|
+-------+---+
|Michael| 29|
|   Andy| 30|
+-------+---+



In [ ]:
DFpeople.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: integer (nullable = true)



###modyfikacja schema - sposob 2

In [ ]:
import pyspark.sql.functions as f

In [ ]:
df = (DFpeople
      .withColumn('Age', f.col('Age').cast(StringType())))

In [ ]:
df.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: string (nullable = true)



###ładowanie df od razu ze zdefiniowana schema

In [ ]:
schema_for_people = StructType([StructField('Names', StringType()),
                                StructField('Age', IntegerType())])

In [ ]:
df_with_schema = spark.read.csv(path='people_noheader.txt', schema=schema_for_people)

In [ ]:
df_with_schema.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: integer (nullable = true)



### DF $\mapsto$ RDD

In [ ]:
df_with_schema.rdd.take(10)

[Row(Names='Michael', Age=None),
 Row(Names='Andy', Age=None),
 Row(Names='Justin', Age=None),
 Row(Names='Filip', Age=None)]

In [ ]:
names = df_with_schema.rdd.map(lambda x: x.Names).collect()

In [ ]:
names

['Michael', 'Andy', 'Justin', 'Filip']

In [ ]:
names = df_with_schema.rdd.map(lambda x: x[0]).collect()

In [ ]:
names

['Michael', 'Andy', 'Justin', 'Filip']

In [ ]:
rdd_with_tuple = df_with_schema.rdd.map(lambda x: (x[0], x[1]))

In [ ]:
rdd_with_tuple.take(5)

[('Michael', None), ('Andy', None), ('Justin', None), ('Filip', None)]

In [ ]:
type(rdd_with_tuple)

pyspark.rdd.PipelinedRDD

##Zadanie 4
1. Załaduj plik pracownicy.csv z poprawnie zdefiniowaną strukturą i typami danych (schema)
2. Dla każdego roku urodzenia występującego w danych wylicz średnią stawkę godzinową
3. posortuj od najmłodszych lat do najstarszych
4. Wynik zapisz do excela

In [ ]:
#kod
pracownicy = spark.read.option('sep', ";").csv('pracownicy.csv', header=True)

In [ ]:
pracownicy.show(10)

+-------+---------+---------------------------+-------------+
|   Imie|Nazwisko |stawka godzinowa w dolarach|rok urodzenia|
+-------+---------+---------------------------+-------------+
| Maciej| Kowalski|                         78|         1944|
|Michael| Kowalski|                         57|         1965|
|    Ewa| Kowalska|                         23|         1999|
|   John|    Nowak|                         12|         2020|
|  Steve| Stevniak|                         67|         1955|
|   Bill|    Gates|                         67|         1955|
|   Jan | Kowalski|                         23|         1999|
|   John|    Nowak|                         12|         2020|
|  Steve| Stevniak|                         67|         1955|
|   Bill|    Gates|                         67|         1955|
+-------+---------+---------------------------+-------------+
only showing top 10 rows



In [ ]:
kolumny = [StructField('Imie', StringType()),
          StructField('Nazwisko', StringType()),
          StructField('stawka godzinowa w dolarach', IntegerType()),
          StructField('rok urodzenia', IntegerType())]

In [ ]:
schema = StructType(kolumny)

In [ ]:
schema

StructType([StructField('Imie', StringType(), True), StructField('Nazwisko', StringType(), True), StructField('stawka godzinowa w dolarach', IntegerType(), True), StructField('rok urodzenia', IntegerType(), True)])

In [ ]:
from pyspark.sql.functions import avg

In [ ]:
stawka = (pracownicy
          .groupBy("rok urodzenia")
          .agg(avg("stawka godzinowa w dolarach").alias("średnia stawka godzinowa"))
          .orderBy("rok urodzenia",ascending=False))

In [ ]:
stawka.show(10)

+-------------+------------------------+
|rok urodzenia|średnia stawka godzinowa|
+-------------+------------------------+
|         2020|                    12.6|
|         1999|                    23.0|
|         1965|                    57.0|
|         1955|                    67.0|
|         1944|                    78.0|
+-------------+------------------------+



In [ ]:
wynik = stawka.toPandas().to_excel("wynik.xlsx")


#Zajęcia 25.04.2022

##UDF

In [5]:
df = spark.createDataFrame([("2022-04-25", "John Doe", 21)], ("date", "name", "age"))
df.show()

+----------+--------+---+
|      date|    name|age|
+----------+--------+---+
|2022-04-25|John Doe| 21|
+----------+--------+---+



In [ ]:
#Jak wyciąć rok z daty?

In [6]:
from pyspark.sql.functions import udf
import pyspark.sql.functions as f
from pyspark.sql.types import *

In [7]:
def f_do_wycinania_roku(x):
  return x[:4]

In [13]:
udf_do_wycinania_roku = udf(f_do_wycinania_roku, StringType())

In [14]:
(df
 .withColumn('date', udf_do_wycinania_roku(f.col('date')))
 ).show()

+----+--------+---+
|date|    name|age|
+----+--------+---+
|2022|John Doe| 21|
+----+--------+---+



In [ ]:
#Jeżeli chcemy UDF wykorzystywać później w SQL to trzeba go zarejestrować (3.3 Registering PySpark UDF & use it on SQL -> https://sparkbyexamples.com/pyspark/pyspark-udf-user-defined-function/)

In [15]:
#""" Using UDF on SQL """
spark.udf.register("udf_do_wycinania_roku", udf_do_wycinania_roku)
df.createOrReplaceTempView("df")
spark.sql("select udf_do_wycinania_roku(date), name, age from df").show()

+---------------------------+--------+---+
|udf_do_wycinania_roku(date)|    name|age|
+---------------------------+--------+---+
|                       2022|John Doe| 21|
+---------------------------+--------+---+



###Pagecounts dataset

In [16]:
from pyspark.sql.types import *
import pyspark.sql.functions as f

In [17]:
!unzip pagecounts.zip

Archive:  pagecounts.zip
   creating: pagecounts/
  inflating: pagecounts/part-00001   
  inflating: pagecounts/part-00053   


In [19]:
!head pagecounts/part-00001

20090505-000000 aa Main_Page 2 9980
20090505-000000 ab %D0%90%D0%B8%D0%BD%D1%82%D0%B5%D1%80%D0%BD%D0%B5%D1%82 1 465
20090505-000000 ab %D0%98%D1%85%D0%B0%D0%B4%D0%BE%D1%83_%D0%B0%D0%B4%D0%B0%D2%9F%D1%8C%D0%B0 1 16086
20090505-000000 af.b Tuisblad 1 36236
20090505-000000 af.d Tuisblad 4 189738
20090505-000000 af.q Tuisblad 2 56143
20090505-000000 af Afrika 1 46833
20090505-000000 af Afrikaans 2 53577
20090505-000000 af Australi%C3%AB 1 132432
20090505-000000 af Barack_Obama 1 23368


In [20]:
!head pagecounts/part-00053

20090507-040000 aa Main_Page 7 51309
20090507-040000 ab %D0%90%D0%B8%D0%BD%D1%82%D0%B5%D1%80%D0%BD%D0%B5%D1%82 1 34069
20090507-040000 ab %D0%98%D1%85%D0%B0%D0%B4%D0%BE%D1%83_%D0%B0%D0%B4%D0%B0%D2%9F%D1%8C%D0%B0 3 65763
20090507-040000 af.b Tuisblad 1 36231
20090507-040000 af.d Tuisblad 1 58960
20090507-040000 af.q Tuisblad 1 44265
20090507-040000 af Afrikaans 3 80838
20090507-040000 af Australi%C3%AB 1 132433
20090507-040000 af Ensiklopedie 2 60584
20090507-040000 af Internet 1 48816


In [21]:
pagecounts = sc.textFile('pagecounts')

In [22]:
pagecounts.count()

1398882

In [23]:
pagecounts.take(10)

['20090507-040000 aa Main_Page 7 51309',
 '20090507-040000 ab %D0%90%D0%B8%D0%BD%D1%82%D0%B5%D1%80%D0%BD%D0%B5%D1%82 1 34069',
 '20090507-040000 ab %D0%98%D1%85%D0%B0%D0%B4%D0%BE%D1%83_%D0%B0%D0%B4%D0%B0%D2%9F%D1%8C%D0%B0 3 65763',
 '20090507-040000 af.b Tuisblad 1 36231',
 '20090507-040000 af.d Tuisblad 1 58960',
 '20090507-040000 af.q Tuisblad 1 44265',
 '20090507-040000 af Afrikaans 3 80838',
 '20090507-040000 af Australi%C3%AB 1 132433',
 '20090507-040000 af Ensiklopedie 2 60584',
 '20090507-040000 af Internet 1 48816']

##Zadanie 5
Jakiego dnia ile było wejść a angielską wikipedię? Rozwiąż na dwa sposoby (wykorzystując czyste RDD oraz Dataframes)

In [43]:
#kod 1 (RDD)
(pagecounts
    .map(lambda x: x.split(' '))
    .filter(lambda x: x[1] == 'en')
    .map(lambda x: [x[0][:8], int(x[3])])
    .reduceByKey(lambda x,y: x+y)
    ).take(10)


[('20090507', 6175726), ('20090505', 7076855)]

In [51]:
#kod 2 (Dataframes)
dataframes = spark.read.csv('pagecounts', sep=' ')

In [69]:
dataframes.show(10)

+---------------+----+--------------------+---+------+
|            _c0| _c1|                 _c2|_c3|   _c4|
+---------------+----+--------------------+---+------+
|20090505-000000|  aa|           Main_Page|  2|  9980|
|20090505-000000|  ab|%D0%90%D0%B8%D0%B...|  1|   465|
|20090505-000000|  ab|%D0%98%D1%85%D0%B...|  1| 16086|
|20090505-000000|af.b|            Tuisblad|  1| 36236|
|20090505-000000|af.d|            Tuisblad|  4|189738|
|20090505-000000|af.q|            Tuisblad|  2| 56143|
|20090505-000000|  af|              Afrika|  1| 46833|
|20090505-000000|  af|           Afrikaans|  2| 53577|
|20090505-000000|  af|      Australi%C3%AB|  1|132432|
|20090505-000000|  af|        Barack_Obama|  1| 23368|
+---------------+----+--------------------+---+------+
only showing top 10 rows



In [65]:
fields =  [StructField('data', StringType()),
          StructField('jezyk', StringType()),
          StructField('strona', StringType()),
          StructField('wyswietlenia', IntegerType()),
          StructField('nie wiem', StringType())]
schema=StructType(fields)

In [66]:
def f_ucinanie(x):
  return x[:8]

In [67]:
udf_do_wycinania_znakow = udf(f_ucinanie, StringType())

In [70]:
dataframes.df = spark.read.csv(path='pagecounts', sep=' ', schema=schema)

In [71]:
(dataframes.df
 .filter(f.col('jezyk') == 'en')
 .select(f.col('data'),f.col('wyswietlenia'))
 .withColumn('data', udf_do_wycinania_znakow(f.col('data')))
 .groupBy('data')
 .sum('wyswietlenia')
).show(10)

+--------+-----------------+
|    data|sum(wyswietlenia)|
+--------+-----------------+
|20090505|          7076855|
|20090507|          6175726|
+--------+-----------------+



##Zadanie 6
1. Wczytaj wiki_sample.parquet (moodle)
2. Odpowiedz na pytanie którzy użytkownicy edytowali najwięcej stron na wikipedii (top 5)?
3. Wykonaj zadanie na dwa sposoby (Dataframe SQL oraz Dataframe programmatic API)

In [ ]:
#kod

#Zajęcia 04.05.2022

##Funkcje agregujące

In [ ]:
df = spark.read.csv('sales_info.csv', inferSchema=True, header=True)

In [ ]:
df.show()

In [ ]:
df.printSchema()

In [ ]:
df.orderBy(['Company', 'Sales']).show()

In [ ]:
df.orderBy([f.col('Company'), f.col('Sales').desc()]).show()

In [ ]:
df.groupBy('Company').max().show()

In [ ]:
df.groupBy('Company').min().show()

In [ ]:
df.groupBy('Company').sum().show()

In [ ]:
(df
 .groupBy('Company')
 .agg(f.min('Sales').alias('MIN'),
      f.max('Sales').alias('MAX'),
      f.avg('Sales'),
      f.stddev('Sales'))
).withColumn('stddev_samp(Sales)', f.round(f.col('stddev_samp(Sales)'), 2)).show()

##Funkcje okienkowe

Sources: https://sparkbyexamples.com/pyspark/pyspark-window-functions/ , https://databricks.com/blog/2015/07/15/introducing-window-functions-in-spark-sql.html

Window functions explained: https://www.youtube.com/watch?v=wH6RaEWuiDQ (polecam!)

In [ ]:
import sys
from pyspark.sql.window import Window

In [ ]:
simpleData = (("James", "Sales", 3000), \
    ("Michael", "Sales", 4600),  \
    ("Robert", "Sales", 4100),   \
    ("Maria", "Finance", 3000),  \
    ("James", "Sales", 3000),    \
    ("Scott", "Finance", 3300),  \
    ("Jen", "Finance", 3900),    \
    ("Jeff", "Marketing", 3000), \
    ("Kumar", "Marketing", 2000),\
    ("Saif", "Sales", 4100) \
  )

In [ ]:
columns= ["employee_name", "department", "salary"]
df = spark.createDataFrame(data = simpleData, schema = columns)
df.printSchema()
df.show(truncate=False)
df.createOrReplaceTempView("df")

###row_number()

In [ ]:
from pyspark.sql.functions import row_number

In [ ]:
windowSpec = (Window
              .partitionBy(f.col('department'))
              .orderBy(f.col('salary').desc())
              )

In [ ]:
df.withColumn('row_number', row_number().over(windowSpec)).show()

In [ ]:
sqlDF = spark.sql("""SELECT employee_name, department, salary,
                     row_number() OVER(PARTITION BY department ORDER BY salary DESC) AS row_number
                     FROM df
""")

sqlDF.show()

###rank()

In [ ]:
from pyspark.sql.functions import rank

In [ ]:
df.withColumn("rank", rank().over(windowSpec)).show()

In [ ]:
sqlDF = spark.sql("""SELECT employee_name, department, salary,
                     rank() OVER(PARTITION BY department ORDER BY salary DESC) AS rank
                     FROM df
""")

sqlDF.show()

###dense_rank()

In [ ]:
from pyspark.sql.functions import dense_rank
df.withColumn("dense_rank", dense_rank().over(windowSpec)).show()

In [ ]:
sqlDF = spark.sql("""SELECT employee_name, department, salary,
                     dense_rank() OVER(PARTITION BY department ORDER BY salary DESC) AS dense_rank
                     FROM df
""")

sqlDF.show()

###min(), max()

In [ ]:
df.withColumn("min_departament", f.min('salary').over(windowSpec.rangeBetween(-sys.maxsize, sys.maxsize))).show()

In [ ]:
sqlDF = spark.sql(""" SELECT employee_name, department, salary,
                      min(salary) OVER(PARTITION BY department ORDER BY salary DESC RANGE BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) as min_salary
                      FROM df
""")
sqlDF.show()

###LAG()

In [ ]:
import pyspark.sql.functions as f

In [ ]:
windowSpec = (Window
              .partitionBy(f.col('department'))
              .orderBy(f.col('salary').desc())
              )

In [ ]:
from pyspark.sql.functions import lag

In [ ]:
df.withColumn("lag_salary", lag("salary", 1).over(windowSpec)).show()

In [ ]:
sqlDF = spark.sql("""SELECT employee_name, department, salary,
                     LAG(salary) OVER(PARTITION BY department ORDER BY salary DESC) as lag_salary
                     FROM df

""")

In [ ]:
sqlDF.show(10)

###LEAD()

In [ ]:
from pyspark.sql.functions import lead

In [ ]:
df.withColumn("lead_salary", lead("salary", 2).over(windowSpec)).show()

In [ ]:
#SQL: https://www.youtube.com/watch?v=tNdJKrwQKIU

##Zadanie 7
1. wczytaj plik bonuses.csv
2. stwórz tabelę o kolumnach employee_id, year, max_quarter_bonus, min_quarter_bonus, total_bonus, total_bonus_previous_year, total_bonus_2_years_ago

In [ ]:
#kod

#Zajęcia 09.05.2022

##Machine Learning

##Colaborative Filtering
Dla chętnych: https://www.youtube.com/watch?v=ZspR5PZemcs

In [ ]:
!unzip ml-100k.zip

In [ ]:
#u.data     -- The full u data set, 100000 ratings by 943 users on 1682 items.
#              Each user has rated at least 20 movies.  Users and items are
#              numbered consecutively from 1.  The data is randomly
#              ordered. This is a tab separated list of
#	         user id | item id | rating | timestamp.
#              The time stamps are unix seconds since 1/1/1970 UTC

In [ ]:
!head -5 ml-100k/u.data

In [ ]:
ratings_data = spark.read.csv("ml-100k/u.data", sep="	", header=False, inferSchema=True)

In [ ]:
from pyspark.sql.types import *

In [ ]:
fields = [StructField('user_id', IntegerType()),
          StructField('item_id', IntegerType()),
          StructField('rating', IntegerType()),
          StructField('timestamp', IntegerType())
          ]

In [ ]:
schema = StructType(fields)

In [ ]:
ratings_data = spark.read.csv("ml-100k/u.data", sep="	", header=False, schema=schema)

In [ ]:
ratings_data.show(10)

In [ ]:
(train, test) = ratings_data.randomSplit([0.8, 0.2])

In [ ]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.sql import Row

In [ ]:
#https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.ml.recommendation.ALS.html#pyspark.ml.recommendation.ALS
als = ALS(maxIter=5, rank=20, regParam=0.01, userCol="user_id", itemCol="item_id", ratingCol="rating", coldStartStrategy="drop")

In [ ]:
model = als.fit(train)

In [ ]:
predictions = model.transform(test)

In [ ]:
predictions.show(10)

In [ ]:
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="prediction")

In [ ]:
rmse = evaluator.evaluate(predictions)

In [ ]:
print(f"Root-mean-square error = {rmse}")

In [ ]:
#top 10 movies for each user
userRecs = model.recommendForAllUsers(10)

In [ ]:
userRecs.show(10, False)

In [ ]:
#top 10 user for each movie
movieRecs = model.recommendForAllItems(10)

In [ ]:
movieRecs.show(10, False)

##Classification
###Decision Trees

In [ ]:
!unzip train.csv.zip

In [ ]:
data = spark.read.csv("train.csv", header=True, inferSchema=True, sep=",")

In [ ]:
data.show(10)

In [ ]:
(train, test) = data.randomSplit([0.8, 0.2])

|y|x1|x2|x3
|-|-|-|-|
|5|1|2|3|
|3|5|6|7|

|y|X_i|
|-|---|
|5|[1,2,3]|
|3|[5,6,7]|

In [ ]:
from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler(inputCols=[x for x in data.columns if x not in ['Id', 'Cover_Type']], outputCol='features')

In [ ]:
df_assemled_preview = assembler.transform(train)

In [ ]:
df_assemled_preview.show(10)

In [ ]:
df_assemled_preview.rdd.take(1)

In [ ]:
from pyspark.ml.feature import StringIndexer
labelIndexer = StringIndexer(inputCol='Cover_Type', outputCol='label').fit(data)

In [ ]:
df_indexed_prview = labelIndexer.transform(train)
df_indexed_prview.show(10)

In [ ]:
from pyspark.ml.classification import DecisionTreeClassifier

In [ ]:
dt = DecisionTreeClassifier(impurity='gini',
                            maxDepth=5,
                            maxBins=32,
                            featuresCol='features',
                            labelCol='label')

In [ ]:
from pyspark.ml import Pipeline

In [ ]:
pipeline = Pipeline(stages=[assembler, labelIndexer, dt])

In [ ]:
model_pipeline = pipeline.fit(train)

In [ ]:
model_pipeline.stages

In [ ]:
treeModel = model_pipeline.stages[2]

In [ ]:
treeModel

In [ ]:
print(treeModel.toDebugString)

In [ ]:
predictions = model_pipeline.transform(test)

In [ ]:
predictions.show(10, False)

In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [ ]:
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName='accuracy')

In [ ]:
accuracy = evaluator.evaluate(predictions)

In [ ]:
print(accuracy)

##Random Forest

##Zadanie 8
Do tego samego zbioru danych stworz model klasyfikacji oparty na Random Forest (https://spark.apache.org/docs/3.1.1/api/python/reference/api/pyspark.ml.classification.RandomForestClassifier.html). Postaraj się zrobić go w formie "pipeline".

In [ ]:
#kod

#Zajęcia 16.05.2022
omowic labelConverter + hyperparameter tuning

##Hyperparameter Tuning

###Random Forest

In [ ]:
train.cache()
test.cache()

In [ ]:
train.count()
test.count()

In [ ]:
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

In [ ]:
paramGrid = (ParamGridBuilder()
  .addGrid(rf.numTrees, [10, 20])
  .addGrid(rf.maxDepth, [30])
  .addGrid(rf.maxBins, [100])
  .addGrid(rf.impurity, ['entropy'])
  .build()
)

In [ ]:
paramGrid

In [ ]:
pipeline = Pipeline(stages=[assembler, labelIndexer, rf, labelConverter])

In [ ]:
crossval = CrossValidator(estimator=pipeline,
                          estimatorParamMaps=paramGrid,
                          evaluator=evaluator,
                          numFolds=3) # use 10 folds in practice

In [ ]:
# Run cross-validation, and choose the best set of parameters.
cvModel = crossval.fit(train)

In [ ]:
cvModel.bestModel

In [ ]:
params = [{p.name: v for p, v in m.items()} for m in cvModel.getEstimatorParamMaps()]
params

In [ ]:
cvModel.avgMetrics

In [ ]:
best_model = cvModel.bestModel

In [ ]:
predictions_test = best_model.transform(test)

In [ ]:
accuracy_test = evaluator.evaluate(predictions_test)

In [ ]:
print(f"Accuracy test: {accuracy_test}")

In [ ]:
predictions_train = best_model.transform(train)
accuracy_train = evaluator.evaluate(predictions_train)
print(f"Accuracy train: {accuracy_train}")

##Zadanie 9
Zaimplementuj hyperparameter tuning dla drzew decyzyjnych. Parametry:

impurity, ["entropy"];
maxDepth, [10, 30];
maxBins, [30, 300, 100];

In [ ]:
#kod

##Clustering
Based on: https://towardsdatascience.com/k-means-clustering-using-pyspark-on-big-data-6214beacdc8b
K-Means explained: https://www.youtube.com/watch?v=4b5d3muPQmA

In [ ]:
!unzip archive.zip

In [ ]:
data_customer=spark.read.csv('CC GENERAL.csv', header=True, inferSchema=True)
data_customer.printSchema()

In [ ]:
data_customer=data_customer.na.drop()

In [ ]:
from pyspark.ml.feature import VectorAssembler
data_customer.columns
assemble=VectorAssembler(inputCols=[
 'BALANCE',
 'BALANCE_FREQUENCY',
 'PURCHASES',
 'ONEOFF_PURCHASES',
 'INSTALLMENTS_PURCHASES',
 'CASH_ADVANCE',
 'PURCHASES_FREQUENCY',
 'ONEOFF_PURCHASES_FREQUENCY',
 'PURCHASES_INSTALLMENTS_FREQUENCY',
 'CASH_ADVANCE_FREQUENCY',
 'CASH_ADVANCE_TRX',
 'PURCHASES_TRX',
 'CREDIT_LIMIT',
 'PAYMENTS',
 'MINIMUM_PAYMENTS',
 'PRC_FULL_PAYMENT',
 'TENURE'], outputCol='features')
assembled_data=assemble.transform(data_customer)
assembled_data.show(2)

In [ ]:
from pyspark.ml.feature import StandardScaler
scale=StandardScaler(inputCol='features',outputCol='standardized')
data_scale=scale.fit(assembled_data)
data_scale_output=data_scale.transform(assembled_data)
data_scale_output.show(2)

In [ ]:
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator
silhouette_score=[]
evaluator = ClusteringEvaluator(predictionCol='prediction', featuresCol='standardized', \
                                metricName='silhouette', distanceMeasure='squaredEuclidean')
for i in range(2,10):

    KMeans_algo=KMeans(featuresCol='standardized', k=i)

    KMeans_fit=KMeans_algo.fit(data_scale_output)

    output=KMeans_fit.transform(data_scale_output)



    score=evaluator.evaluate(output)

    silhouette_score.append(score)

    print("Silhouette Score:",score)

In [ ]:
#Visualizing the silhouette scores in a plot
import matplotlib.pyplot as plt
fig, ax = plt.subplots(1,1, figsize =(8,6))
ax.plot(range(2,10),silhouette_score)
ax.set_xlabel('k')
ax.set_ylabel('cost')

In [ ]:
KMeans_algo=KMeans(featuresCol='standardized', k=3)
KMeans_fit=KMeans_algo.fit(data_scale_output)
output=KMeans_fit.transform(data_scale_output)
output.show(10)

In [ ]:
from pyspark.ml.feature import PCA
import matplotlib.pyplot as plt

pca = PCA(k=3, inputCol="features", outputCol="pcaFeatures")
model = pca.fit(data_scale_output)

result = model.transform(data_scale_output).select("pcaFeatures")
pandasDf = result.toPandas()
dataX = []
dataY = []

for vec in pandasDf.values:
  dataX.extend([vec[0][0]])
  dataY.extend([vec[0][1]])
plt.scatter(dataX, dataY)
plt.show()

#Zajęcia 23.05.2022

##Data Science How-To: Using Apache Spark for Sports Analytics
based on: https://tanzu.vmware.com/content/blog/how-data-science-assists-sports

In [ ]:
!unzip data.zip

In [ ]:
df = spark.read.option('header','true')\
          .option('inferSchema','true')\
          .csv('data/season_totals.csv')

In [ ]:
df.limit(5).show()

In [ ]:
import pyspark.sql.functions as f

In [ ]:
df.orderBy(f.col('pts').desc()).limit(10).show()

In [ ]:
#Ile było prób rzutu za 3 punkty na 36 minut

In [ ]:
fga_py = (df
          .groupBy('yr')
          .agg(f.sum('mp'), #minutes played (wszystkie rozegrane minuty wszystkich graczy)
               f.sum('fg3a')) #attempts
          .select(f.col('yr'), (36*f.col('sum(fg3a)')/f.col('sum(mp)')).alias('fg3a_p36m'))
          .orderBy('yr')
               )
fga_py.show(10)

In [ ]:
from matplotlib import pyplot as plt
import seaborn as sns

sns.set_style("white")
sns.set_color_codes()

plt.figure(figsize=(20,10))
plt.style.use('fivethirtyeight')

_df = fga_py.toPandas()
plt.plot(_df.yr,_df.fg3a_p36m, color = '#00a79c')
plt.xlabel('Year')
plt.title('Player average 3-point attempts (per 36 minutes)')
plt.annotate('3 pointer introduced', xy=(1980, .5), xytext=(1981, 1.1), fontsize = 12,
            arrowprops=dict(facecolor='grey', shrink=0.05, linewidth = 2))
plt.annotate('NBA moved in 3-point line', xy=(1996, 2.4), xytext=(1991.5, 2.7), fontsize = 12,
            arrowprops=dict(facecolor='grey', shrink=0.05, linewidth = 2))
plt.annotate('NBA moved back\n3-point line', xy=(1998, 2.), xytext=(1998.5, 2.4), fontsize = 12, arrowprops=dict(facecolor='grey', shrink=0.05, linewidth = 2))
plt.show()

###Zadanie 10
Regresja liniowa w Sparku - ile będzie rzutów na koniec 2022? Użyj Regresji Logistycznej

In [ ]:
#kod

###Heatmap 3-point shoots

In [ ]:
#statystyki skąd rzuty
df = spark.read.csv(path='data/shot_charts_top_10/1000_plus_shot_charts_2011_2016.csv', header=True, inferSchema=True)

In [ ]:
df.cache()
df.count()

In [ ]:
df.show()

In [ ]:
from nba_utils import draw_3pt_piechart,plot_shot_chart
import numpy as np
player = 'Stephen Curry'
yr = '2016'
df_steph = df.filter(f'''name == "{player}"
                        and yr == {yr}
                        and y < 400''')
#https://www.programiz.com/python-programming/list-comprehension

x = np.array([v[0] for v in df_steph.select('x').collect()])
y = np.array([v[0] for v in df_steph.select('y').collect()])
p=plot_shot_chart(x, y, gridsize = 30,
                      kind='hex')
# p.savefig('results/steph_curry_2016_shotchart.png')
plt.show()

In [ ]:
import warnings
warnings.simplefilter("ignore")

In [ ]:
df.show(5)

In [ ]:
def is_corner_3(xy):
    '''Want to identify corner 3 point attempts'''
    x,y = xy
    return int((abs(x) >= 220) and (y < 92.5))

def is_normal_3(xycorner3):
    '''Want to identify normal (not corner 3) point attempts'''
    x,y,corner3 = xycorner3
    radius = 475/2.
    y_3pt = np.sqrt(np.square(radius) - np.square(x))
    return int(y > max(92.5,y_3pt) and not corner3)

In [ ]:
from pyspark.sql.functions import array,udf
from pyspark.sql.types import IntegerType

corner_3_udf = udf(is_corner_3, IntegerType())
normal_3_udf = udf(is_normal_3, IntegerType())

df2 = df.withColumn('corner_3', corner_3_udf(array([df.x,df.y])))
df3 = df2.withColumn('normal_3', normal_3_udf(array([df2.x,df2.y,df2.corner_3])))
df4 = df3.withColumn('is_a_3', df3.corner_3 + df3.normal_3)

df = df4
df.cache()

In [ ]:
from pyspark.sql.functions import mean
from nba_utils import draw_3pt_piechart,plot_shot_chart

# make shot charts for all years
midrange_thresh = 8
in_half_court = 'y <= 400 and abs(x) <= 250'
addl_filter = f'shot_distance > {midrange_thresh}'
for yr in range(2011,2016+1):
    df_yr = df.filter(f'''{in_half_court}
                         and yr == {yr}
                         and {addl_filter}''')
    x = np.array([v[0] for v in df_yr.select('x').collect()])
    y = np.array([v[0] for v in df_yr.select('y').collect()])
    p = plot_shot_chart(x,y, gridsize = 30, kind = 'hex', label = yr)
    p.fig.suptitle('Evolution of the 3 point shot', x = .19, y = 0.86, size = 20, fontweight= 'bold')

    per_3 = df_yr.select(mean(df_yr.is_a_3)).take(1)[0][0]
    per_midrange = 1 - per_3
    draw_3pt_piechart(per_3, per_midrange)
    plt.show()
#     p.savefig('results/all_years/{}.png'.format(yr))
#     plt.close()

###Shots efficiency

In [ ]:
from pyspark.sql.functions import count, when

shot_acc = df.groupBy('shot_distance','corner_3','normal_3','is_a_3')\
             .agg(count('*').alias('num_attempts'),mean(df.shot_made_flag).alias('shot_accuracy'))\
             .withColumn('points_per_shot',when(f.col('is_a_3') == 1, f.col('shot_accuracy')*3)
                                           .otherwise(f.col('shot_accuracy')*2)
                        )\
             .filter('num_attempts > 5')\
             .orderBy('shot_distance')\
             .toPandas()

In [ ]:
df.show(3)

In [ ]:
shot_acc

In [ ]:
plt.style.use('fivethirtyeight')
plt.figure(figsize = (10,6))

def plot_acc_vs_dist(df,kwargs = {}):
    plt.plot(df.shot_distance, df.points_per_shot, **kwargs)

plot_acc_vs_dist(shot_acc.query('is_a_3 == False'), {'color' : '#008fd5'})
plot_acc_vs_dist(shot_acc.query('is_a_3 == True'), {'color' : '#008fd5'})
plt.title('Shot value vs. shot distance, 2011-2016 seasons\n Players with 1000+ attempts in a season')
plt.xlim(0,30)
plt.xlabel('Shot Distance (ft)')
plt.ylabel('Points per shot')
plt.annotate('high efficiency 2s', xy=(2., 1.15), xytext=(4.5, 1.28),
            arrowprops=dict(facecolor='grey', shrink=0.05))
plt.annotate('high efficiency 3s', xy=(22, 1.15), xytext=(14.5, 1.15),
            arrowprops=dict(facecolor='grey', shrink=0.05))
plt.text(22, 1.25, 'corner 3s', fontsize = 12)
#plt.savefig('results/pps.png')